In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 1
setup_one_gpu(GPU)

Picking GPU 1


In [10]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from functools import partial

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter
from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params

from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

from tf_lab.iclr.helper import load_multiple_version_of_pcs, find_best_validation_epoch_from_train_stats
from tf_lab.point_clouds.convenience import reconstruct_pclouds

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
experiment_tag = 'mlp_with_split_1pc_usampled_bnorm_on_encoder_only'
loss = 'emd'
n_pc_points = 2048
random_seed = 42
bneck = 128
class_name = raw_input('Give me the class type: ').lower()

Give me the class type: chair


In [5]:
syn_id = snc_category_to_synth_id()[class_name]
in_data = load_multiple_version_of_pcs('uniform_one' ,syn_id, n_classes=1, n_pc_points=2048)

679 pclouds were loaded. They belong in 1 shape-classes.
5761 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.


In [11]:
train_params = default_train_params()
experiment_id = '_'.join(['ae', class_name, experiment_tag, str(n_pc_points), 'pts', str(bneck), 'bneck', loss])
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/', experiment_id)
conf = Conf.load(osp.join(train_dir, 'configuration'))
val_error, best_epoch = find_best_validation_epoch_from_train_stats(osp.join(train_dir, 'train_stats.txt'))
if best_epoch % conf.saver_step != 0: # Model was not saved at that epoch: use next bigger epoch.
    epoch_to_load = best_epoch + (best_epoch % conf.saver_step)
else:
    epoch_to_load = best_epoch
    
conf.encoder_args['verbose'] = False
conf.decoder_args['verbose'] = False
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)    
ae.restore_model(conf.train_dir, epoch_to_load, verbose=True)
_, l_val = reconstruct_pclouds(ae, in_data['val'].point_clouds, batch_size=100)    
_, l_test = reconstruct_pclouds(ae, in_data['test'].point_clouds, batch_size=100)
_, l_train = reconstruct_pclouds(ae, in_data['train'].point_clouds, batch_size=100)

INFO:tensorflow:Restoring parameters from /orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/ae_chair_mlp_with_split_1pc_usampled_bnorm_on_encoder_only_2048_pts_128_bneck_emd/models.ckpt-400
Model restored in epoch 400.


In [13]:
# esto oti exo gia kathe mia karekla ton kontinotero tis geitona
# tora thelo

88.212193547841309

In [15]:
from tf_lab.point_clouds.autoencoder import AutoEncoder

class PCAE_PW_Reg(AutoEncoder):

